In [1]:
from urllib.parse import urlencode
from urllib.parse import quote
import requests
import pandas as pd
import json
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [2]:
file_path = 'city_ids/history.city.list.json'

with open(file_path, 'r', encoding='utf-8') as file:
    city_ids = json.load(file)




In [3]:
city_ids_list = []
for city_id in city_ids:
    a = int(city_id['city']['id']['$numberLong'])
    city_ids_list.append(a)

In [4]:
def getting_historical_data(city_id,start_time):
    api_key = '1eb23228a41c738ff1b4596d76f41a2d'
    endpoint = f'https://history.openweathermap.org/data/2.5/history/city'
    params = {'id': (city_id), 'type' : 'hour','start':start_time,'end':start_time + 60 * 60, 'appid': api_key }
    url_params = urlencode(params)
    url = f'{endpoint}?{url_params}'




    r = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    r.mount('http://', adapter)
    r.mount('https://', adapter)

    r = r.get(url)
   
    if r.status_code not in range(200,299):
        return{}
    weather_data = r.json()
    try:# Extract the required information
        cloudiness = weather_data["list"][0]['clouds']['all']
        weather_main = weather_data["list"][0]['weather'][0]['main']
        weather_description = weather_data["list"][0]['weather'][0]['description']
        temperature = weather_data['list'][0]['main']['temp']
        feels_like = weather_data['list'][0]['main']['feels_like']

        # Return the extracted information as a dictionary
        return {
            "Temperature": temperature,
            "Feels_Like": feels_like,
            "Cloudiness": cloudiness,
            "Weather_Main": weather_main,
            "Weather_Description": weather_description
        }
    except (json.JSONDecodeError):
        return None    

In [5]:
start_time = 1679750400
sity_id = city_ids_list[:30000:2]

In [6]:

weather_info_list = []
try:
    for city_id in sity_id:
        weather_info_list.append(getting_historical_data(city_id,start_time))
except:
        pass
df = pd.DataFrame(weather_info_list)
df.head()

,Temperature,Feels_Like,Cloudiness,Weather_Main,Weather_Description
0,287.72,286.28,86,Clouds,overcast clouds
1,292.19,290.72,20,Clouds,few clouds
2,293.51,292.41,3,Clear,clear sky
3,287.11,285.97,96,Rain,light rain
4,302.91,305.91,100,Clouds,overcast clouds


In [7]:
df.to_csv('final_data.csv')